In [4]:
import pandas as pd
import glob

def read_kma_csv(file):
    header_row = None

    with open(file, encoding="euc-kr") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            clean_line = line.strip().replace('\ufeff', '')
            if clean_line.startswith("지점번호"):
                header_row = i
                break

    if header_row is None:
        raise ValueError(f"{file}에서 헤더를 찾지 못했습니다.")

    df = pd.read_csv(
        file,
        encoding="euc-kr",
        skiprows=header_row
    )

    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.replace('\ufeff','')

    return df


# 🔥 파일 불러오기
file_list = sorted(glob.glob("seoul_d_data*.csv"))

df_list = [read_kma_csv(file) for file in file_list]

merged_df = pd.concat(df_list, ignore_index=True)

print("컬럼 확인:", merged_df.columns)  # ⭐ 여기 먼저 확인

merged_df = merged_df.sort_values("일시")

merged_df.to_csv("seoul_2023_2024_merged.csv",
                 index=False,
                 encoding="utf-8-sig")

print("세로 병합 완료")

ValueError: seoul_d_data1.csv에서 헤더를 찾지 못했습니다.

In [8]:
import pandas as pd
import glob

def read_kma_csv(file):
    header_row = None

    with open(file, encoding="euc-kr") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            clean_line = line.strip().replace('\ufeff','')
            clean_line = clean_line.lstrip("\t")   # 🔥 탭 제거
            if "지점번호" in clean_line and "일시" in clean_line:
                header_row = i
                break

    if header_row is None:
        raise ValueError(f"{file}에서 헤더를 찾지 못했습니다.")

    df = pd.read_csv(
        file,
        encoding="euc-kr",
        skiprows=header_row,
        sep=","
    )

    # 컬럼 정리
    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.replace('\ufeff','')

    return df


# 🔥 파일 불러오기
file_list = sorted(glob.glob("seoul_d_data*.csv"))

df_list = [read_kma_csv(file) for file in file_list]

merged_df = pd.concat(df_list, ignore_index=True)

merged_df["일시"] = pd.to_datetime(merged_df["일시"])
merged_df = merged_df.sort_values("일시")

merged_df.to_csv("seoul_2023_2024_merged.csv",
                 index=False,
                 encoding="utf-8-sig")

print("세로 병합 완료")
print(merged_df.head())

세로 병합 완료
         지점번호 지점명         일시  일조합(hr)  일조율(%)  일사합(MJ/m2)  평균기온(℃)  최고기온(℃)  \
0     \t\t108  서울 2023-01-01      9.0    94.7       10.81      NaN      NaN   
733   \t\t108  서울 2023-01-01      NaN     NaN         NaN     -0.2      3.8   
1472  \t\t108  서울 2023-01-01      NaN     NaN         NaN      NaN      NaN   
1     \t\t108  서울 2023-01-02      9.1    95.8       11.63      NaN      NaN   
734   \t\t108  서울 2023-01-02      NaN     NaN         NaN     -4.5     -0.4   

     최고기온시각  최저기온(℃) 최저기온시각일교차  Unnamed: 8  평균습도(%rh)  최저습도(%rh)  
0       NaN      NaN       NaN         NaN        NaN        NaN  
733   13:43     -4.3     23:50         8.1        NaN        NaN  
1472    NaN      NaN       NaN         NaN       55.0       28.2  
1       NaN      NaN       NaN         NaN        NaN        NaN  
734   15:25     -7.4      8:04         7.0        NaN        NaN  


In [17]:
import pandas as pd
import glob
from functools import reduce

# 🔥 CSV 읽기 함수 (헤더 자동 탐지 + 탭 제거)
def read_kma_csv(file):
    header_row = None

    with open(file, encoding="euc-kr") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            clean_line = line.strip().replace('\ufeff','')
            clean_line = clean_line.lstrip("\t")
            if "지점번호" in clean_line and "일시" in clean_line:
                header_row = i
                break

    if header_row is None:
        raise ValueError(f"{file}에서 헤더를 찾지 못했습니다.")

    df = pd.read_csv(
        file,
        encoding="euc-kr",
        skiprows=header_row,
        sep=","
    )

    # 🔥 컬럼 정리
    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.replace('\ufeff','')

    # 🔥 문자열 컬럼 탭 제거
    df = df.apply(lambda x: x.str.replace("\t","").str.strip()
                  if x.dtype == "object" else x)

    # 🔥 빈 지점번호 제거
    df = df[df["지점번호"] != ""]

    return df


# 🔥 파일 불러오기
file_list = sorted(glob.glob("seoul_d_data*.csv"))

df_list = [read_kma_csv(file) for file in file_list]

# 🔥 날짜 기준 가로 병합
merged_df = reduce(
    lambda left, right: pd.merge(left, right, on=["일시","지점번호","지점명"], how="outer"),
    df_list
)


# 🔥 날짜 정리
merged_df["일시"] = pd.to_datetime(merged_df["일시"])
merged_df = merged_df.sort_values("일시").reset_index(drop=True)

# 🔥 저장
merged_df.to_csv("seoul_2023_2024_final.csv",
                 index=False,
                 encoding="utf-8-sig")

print("가로 병합 완료")
print(merged_df.head())

가로 병합 완료
  지점번호 지점명         일시  일조합(hr)  일조율(%)  일사합(MJ/m2)  평균기온(℃)  최고기온(℃) 최고기온시각  \
0  108  서울 2023-01-01      9.0    94.7       10.81     -0.2      3.8  13:43   
1  108  서울 2023-01-02      9.1    95.8       11.63     -4.5     -0.4  15:25   
2  108  서울 2023-01-03      9.1    95.8       11.77     -5.0      0.6  15:55   
3  108  서울 2023-01-04      8.7    91.6       10.89     -1.8      3.3  15:10   
4  108  서울 2023-01-05      4.7    48.5        6.09     -1.6      3.6  15:36   

   최저기온(℃) 최저기온시각일교차  Unnamed: 8  평균습도(%rh)  최저습도(%rh)  
0     -4.3     23:50         8.1       55.0       28.2  
1     -7.4      8:04         7.0       46.0       31.9  
2     -9.0      5:51         9.6       49.0       27.0  
3     -5.7      2:29         9.0       51.0       33.7  
4     -5.6      7:49         9.2       58.1       39.0  
